# Movebank API ingestion

Small example on Movebank data querying and transformation.

In [ ]:
import requests # HTTP library
import base64
import json
import fiona
import time, datetime

# Some useful global variables:
movebank_api_root = "https://www.movebank.org/movebank/service/public/json"
credentials = input().encode('ascii')

movebank_api_headers = {
    "Content-Type": "application/json",
    "User-Agent": "Python3.6 requests",
    "Authorization": f"Basic: {base64.b64encode(credentials)}"
}

# 👇 Remove the output below! DON'T COMMIT!

In [ ]:
# Get tracking data for a list of individuals in a single study
def get_raw_tracking_data(study_id, individual_ids, headers = movebank_api_headers):
    # Convert to list if only a single individual is selected:
    if type(individual_ids) is not list: individual_ids = [individual_ids]
    individual_params = "&".join([f"individual_local_identifiers[]={iid}" for iid in individual_ids])
    final_url = f"{movebank_api_root}?study_id={study_id}&{individual_params}&sensor_type=gps"
    req = requests.get(final_url, headers = headers)
    res = json.loads(req.text)
    return res['individuals']

raw_tracking_data = get_raw_tracking_data("2911040", ["4262-84830876", "1163-1163"])
# (raw_tracking_data[0]) # We'll have a list of stuff like this, one per individual
# raw_tracking_data

In [ ]:
def gps_to_utc(gps_time, decimals):
    gps_time = str(gps_time)
    len_time= len(gps_time)
    split_gps_time = (
        "".join(list(gps_time)[:(len_time - decimals)]),
        "".join(list(gps_time)[(len_time - decimals):])
    )
    joined = float(".".join(split_gps_time))
    return time.strftime("%b %d %Y %H:%M:%S", time.gmtime(joined))

def transform_raw_tracking_data(raw_tracking_data):
    positions = raw_tracking_data['locations']
    result = [{
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [
                position['location_long'], # Standard GeoJSON order
                position['location_lat'],  # Z coord can be added
            ]
        },
        "properties": {"timestamp": gps_to_utc(position['timestamp'], 3)}
    } for position in positions]
    return result

with open("output.json", "w") as geojson:
    json.dump(
        {
            "type": "FeatureCollection",
            "features": transform_raw_tracking_data(raw_tracking_data[0])
        },
        geojson
    )

# {"type": "FeatureCollection", "features": transform_raw_tracking_data(raw_tracking_data[0])}
    